In [9]:
# Import Python Libraries
import pickle
import h5py
import numpy as np
import xarray as xr
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import copy
from collections import OrderedDict
import random
import glob

In [10]:
# ERA5 calculation functions

# Function to get 2D variable data
def calc_2D_var(data_path,lati,loni,datei):

    data = xr.open_dataset(data_path)

    varname = list(data.keys())[0]
    data = data[varname]
    
    # Get time index (idt) within each file
    diff_time = pd.to_datetime(data.time.values)-date_i
    idt = np.where(np.abs(diff_time) == np.min(np.abs(diff_time)))

    # Get lon index within each file
    diff_lon = data.longitude.values - loni
    idlon = np.where(np.abs(diff_lon) == np.min(np.abs(diff_lon)))

    # Get lat index within each file
    diff_lat = data.latitude.values - lati
    idlat = np.where(np.abs(diff_lat) == np.min(np.abs(diff_lat)))

    var = data[int(idt[0]),int(idlat[0]),int(idlon[0])].values
    
    return var
            
# Function to calculate the mean specific humidity between two pressure levels
def calc_mean_layer_sh(alt_arr,data_path,lati,loni,datei):

    # Open up data with xarray
    data = xr.open_dataset(data_path)

    # Get time index (idt) within each file
    diff_time = pd.to_datetime(data.time.values)-datei
    idt = np.where(np.abs(diff_time) == np.min(np.abs(diff_time)))
            
    # Get lon index within each file
    diff_lon = data.longitude.values - loni
    idlon = np.where(np.abs(diff_lon) == np.min(np.abs(diff_lon)))

    # Get lat index within each file
    diff_lat = data.latitude.values - lati
    idlat = np.where(np.abs(diff_lat) == np.min(np.abs(diff_lat)))

    # Get pressure level indicies
    alt0 = np.where(np.abs(np.array(data.level[:].values) - alt_arr[0]) == np.min(np.abs(np.array(data.level[:].values) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(data.level[:].values) - alt_arr[1]) == np.min(np.abs(np.array(data.level[:].values) - alt_arr[1])))
    #if n == 0:
    #    print(alt0[0],alt1[0])

    # Get specific data and do calculation
    var1 = data.q[int(idt[0]),int(alt0[0]):int(alt1[0]),int(idlat[0]),int(idlon[0])]
    var = np.nanmean(var1)
    del(var1)

    data.close()

    return var
        
# Function to calculate the mean lapse rate between two pressure levels ( K/km )
def calc_mean_lapse_rate(alt_arr,t_data_path,z_data_path,lati,loni,datei):

    g = 9.807 #m/s^2

    # Open up data with xarray
    tdata = xr.open_dataset(t_data_path)
    zdata = xr.open_dataset(z_data_path)

    
    # Get time index (idt) within each file
    diff_time = pd.to_datetime(tdata.time.values)-datei
    idt = np.where(np.abs(diff_time) == np.min(np.abs(diff_time)))    
    
    # Get lon index within each file
    diff_lon = tdata.longitude.values - loni
    idlon = np.where(np.abs(diff_lon) == np.min(np.abs(diff_lon)))

    # Get lat index within each file
    diff_lat = tdata.latitude.values - lati
    idlat = np.where(np.abs(diff_lat) == np.min(np.abs(diff_lat)))

    # Get pressure level indicies
    alt0 = np.where(np.abs(np.array(tdata.level[:].values) - alt_arr[0]) == np.min(np.abs(np.array(tdata.level[:].values) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(tdata.level[:].values) - alt_arr[1]) == np.min(np.abs(np.array(tdata.level[:].values) - alt_arr[1])))
    #if n == 0:
    #    print(alt0[0],alt1[0])

    # Get specific data and do calculation
    var1 = tdata.t[int(idt[0]),int(alt0[0]),int(idlat[0]),int(idlon[0])]
    var2 = tdata.t[int(idt[0]),int(alt1[0]),int(idlat[0]),int(idlon[0])]

    var3 = zdata.z[int(idt[0]),int(alt0[0]),int(idlat[0]),int(idlon[0])]
    var4 = zdata.z[int(idt[0]),int(alt1[0]),int(idlat[0]),int(idlon[0])]

    var = (var1-var2)/((var3-var4)/g/1000)
    del(var1,var2,var3,var4)

    tdata.close()
    zdata.close()

    return var
    

# Function to calculate the wind shear between two pressure levels
def calc_layer_shear(alt_arr,data_path_u,data_path_v,lati,loni,datei):

    #### LOW LEVEL SHEAR        
#    udata = xr.open_dataset(inpath+upath+era5_datestr+'_'+hemi+'.nc')
#    vdata = xr.open_dataset(inpath+vpath+era5_datestr+'_'+hemi+'.nc')
    udata = xr.open_dataset(data_path_u)
    vdata = xr.open_dataset(data_path_v)

    # Get time index (idt) within each file
    diff_time = pd.to_datetime(udata.time.values)-datei
    idt = np.where(np.abs(diff_time) == np.min(np.abs(diff_time)))    

    # Get lon index within each file
    diff_lon = udata.longitude.values - loni
    idlon = np.where(np.abs(diff_lon) == np.min(np.abs(diff_lon)))

    # Get lat index within each file
    diff_lat = vdata.latitude.values - lati
    idlat = np.where(np.abs(diff_lat) == np.min(np.abs(diff_lat)))   
    
#    alt_arr = [800, 1000]
    alt0 = np.where(np.abs(np.array(udata.level[:].values) - alt_arr[0]) == np.min(np.abs(np.array(udata.level[:].values) - alt_arr[0])))
    alt1 = np.where(np.abs(np.array(udata.level[:].values) - alt_arr[1]) == np.min(np.abs(np.array(udata.level[:].values) - alt_arr[1])))
    if n == 0:
        print(alt0[0],alt1[0])

    var1 = udata.u[int(idt[0]),int(alt0[0]),int(idlat[0]),int(idlon[0])]
    var2 = vdata.v[int(idt[0]),int(alt0[0]),int(idlat[0]),int(idlon[0])]
    var3 = udata.u[int(idt[0]),int(alt1[0]),int(idlat[0]),int(idlon[0])]
    var4 = vdata.v[int(idt[0]),int(alt1[0]),int(idlat[0]),int(idlon[0])]
    var = np.sqrt(np.power((var3-var1),2.0) + np.power((var4-var2),2.0))
    #lls_vals = np.append(lls_vals,var)       
    
    del(var1,var2,var3,var4)
    udata.close()
    vdata.close()
    
    return var
    
#def read_IMERG_CT(filename)
    
    

In [15]:
#Read in dataset from Bowen
BP_path = '/home/pmarin/INCUS/Code/IMERG/IMERG_CT_BP/IMERG_CT_Analyses/BP_Data_2007/'
BPfiles = glob.glob(BP_path+'*csv')

BPdata = pd.DataFrame()
for i in np.arange(0,len(BPfiles)):
    BPcur = pd.read_csv(BPfiles[i])
    BPdata = pd.concat([BPdata, BPcur],ignore_index=True)


In [16]:
BPdata

,JDate,Granuel,STID,UTC Time,Day or Night,MeanFlag,Pedestal_LON,Pedestal_LAT,Longitude,Latitude
0,32,4060,1,4,0,2.00,-37.84,-47.32,-37.73,-47.02
1,32,4060,12,4,1,1.11,146.94,-22.22,147.15,-23.04
2,32,4060,25,4,1,1.15,142.78,-3.51,142.78,-3.49
3,32,4060,34,4,1,2.00,141.06,4.59,140.98,4.95
4,32,4061,0,5,0,1.74,-50.81,-2.38,-50.71,-1.89
...,...,...,...,...,...,...,...,...,...,...
20143,90,4904,28,3,1,2.00,156.45,4.76,156.45,4.74
20144,90,4904,43,3,1,2.00,155.44,9.48,155.54,9.02
20145,90,4905,19,5,1,1.19,133.11,-1.72,133.13,-1.83
20146,90,4905,19,5,1,2.00,132.60,0.66,132.65,0.44


In [9]:
# Specific .mat files that has information of IMERG_CT tracks
#matpaths = ['/home/pmarin/INCUS/Code/IMERG/IMERG_CT_20070112.mat',
#            '/home/pmarin/INCUS/Code/IMERG/IMERG_CT_20080112.mat']
#matpaths = ['/home/pmarin/INCUS/Code/IMERG/New_IMERG_CT_2007.mat']
matpaths = ['/avalanche/pmarin/INCUS/IMERG-CT/IMERG_data2_with_HXG_MERIR_2007.nc']

In [15]:
# Read in a list of the IMERG-CT Dataset Variables
arrays = {}
f = h5py.File(matpaths[i])
for k, v in f.items():
    arrays[k] = np.array(v)

In [27]:
# Specifiy Specific Variables from IMERG-CT Dataset to read-in
# Family / Convective System
vars_in = ['CS_id','CS_year','CS_month','CS_day','CS_hour','CS_life','frac_ocean',
           'max_lat','min_lat','max_lon','min_lon','minCTT_MERGED','CS_minminCTT_MERGED','maxP','CS_maxmaxp','area','area_low']
# Family 
# area_low (total area of system based on precitation rate of > 0.5 mm/hr)
# area (convective system area precipitation 5 mm/hr)

In [28]:
# Read in specific variables (vars_in) from the IMERG-CT and convert to pandas DF
data_in = OrderedDict()
for v in np.arange(0,len(vars_in)):
    data_in[vars_in[v]] = arrays[vars_in[v]][0]
    
df_CT = pd.DataFrame.from_dict(data_in)

In [29]:
# Get List of unique identifiers
uni_ids = np.unique(df_CT['CS_id'])

In [30]:
# Open file with xarray
#cdata = xr.open_dataset(inpath+cpath+era5_datestr+'.nc')
#cdata = xr.open_dataset(inpath+cpath+era5_datestr+'_'+hemi+'.nc')

# Get time index (idt) within each file
#diff_time = pd.to_datetime(cdata.time.values)-date_i
#idt = np.where(diff_time == np.min(np.abs(diff_time)))

# Get lon index within each file
#diff_lon = cdata.longitude.values - loni
#idlon = np.where(diff_lon == np.min(np.abs(diff_lon)))

# Get lat index within each file
#diff_lat = cdata.latitude.values - lati
#idlat = np.where(diff_lat == np.min(np.abs(diff_lat)))

#cdata.cape['time']

In [31]:
#inpath = '/avalanche/pmarin/INCUS/ERA5/GEO/GEO_2007_12_SH.nc'
#test = xr.open_dataset(inpath)
#print(test)

#inpath = '/avalanche/pmarin/INCUS/ERA5/FL/FL_2007_12.nc'
#test = xr.open_dataset(inpath)
#print(test)

In [287]:
# Create an hourly list of dates for the entire year 2007
numhrs = 8760
base = datetime.datetime(2007,1,1,0)
date_list = [base + datetime.timedelta(hours=x) for x in range(numhrs)]

In [24]:
# Pull colocated data from ERA5

rand_on = 0 # if 1, choose random lat/lon/time 
saveadd = 'rand' #Additional save name

inpath = '/avalanche/pmarin/INCUS/ERA5/' # ERA5 datapaths

pickpath = '/avalanche/pmarin/INCUS/IMERG-CT/pickle/' # Save pickle file path

# Lat Lon Thresholds
lat_arr = [-20, 20]
lon_arr = [-180, 180]

len_thr = 3 # Atleast three time steps long (i.e., 1 hour)

comb_data = OrderedDict()
data_all = []
for i in np.arange(0,1):
    
    # Filename to save
    filename = 'IMERG-CT_ERA5_LAT'+str(lat_arr[0])+str(lat_arr[1])+'_LON'+str(lon_arr[0])+str(lon_arr[1])+saveadd+'.p'

    # Loop through unique objects and find ERA5 variables
    for n in np.arange(0,len(uni_ids),1):
        print('n=',n)
        #    for n in np.arange(0,2,1):
        fam1 = uni_ids[n]
        #print(fam1)
        
        # Grab dataframe associated with current family
        cur_df = df_CT[df_CT['CS_id'] == uni_ids[n]]
        df_0 = cur_df.head(1)
        #print(n,df_0)
        
        # Threshold families that last at least len_thr time steps
        if df_0['CS_life'].iloc[0] < len_thr:
            continue

        # Get average lat / lon at initial time from max and min lat and lon values
        if rand_on == 1:
            lati = random.choice(np.arange(lat_arr[0]*100,lat_arr[1]*100,5)/100)
        else:
            lati = (df_0['max_lat'].iloc[0] + df_0['min_lat'].iloc[0])/2.0
        df_0['lati'] = lati
        
        if rand_on == 1:
            loni = random.choice(np.arange(lon_arr[0]*100,lon_arr[1]*100,5)/100)
        else:
            loni = (df_0['max_lon'].iloc[0] + df_0['min_lon'].iloc[0])/2.0
        df_0['loni'] = loni
               
        # Exclude events at latitudes greater than |35|
        if np.abs(lati) > 35:
            continue

        # Define NH or SH for filenaming convention for ERA5 files
        if lati >= 0:
            hemi = 'NH'
        else:
            hemi = 'SH'  

        # Current date and time of initial precipitation detection of IMERG-CT families
        if rand_on == 1:
            date_i = random.choice(date_list)
        else:
            date_i = datetime.datetime(int(df_0['CS_year'].iloc[0]),int(df_0['CS_month'].iloc[0]),int(df_0['CS_day'].iloc[0]),int(df_0['CS_hour'].iloc[0]))
        
        # Look at certain time (i.e., 2 hours before detection)
        date_i = date_i+datetime.timedelta(hours=0)
        if date_i.year == 2006: # Only 2007/2008 data
            continue
#        print(date_i)


        # Format for finding the correct era5 file
        era5_datestr = date_i.strftime("%Y_%m")   

        # Low Level Wind Shear
        alt_arr = [800, 1000]
        upath = inpath+'U/U_'+era5_datestr+'_'+hemi+'.nc'
        vpath = inpath+'V/V_'+era5_datestr+'_'+hemi+'.nc'
        var0 = calc_layer_shear(alt_arr,upath,vpath,lati,loni,date_i)
        df_0['llws'] = copy.deepcopy(np.round(var0.values,4))
        del(var0,alt_arr,upath,vpath)
        
        # Mid Level Wind Shear
        alt_arr = [400, 800]
        upath = inpath+'U/U_'+era5_datestr+'_'+hemi+'.nc'
        vpath = inpath+'V/V_'+era5_datestr+'_'+hemi+'.nc'
        var0 = calc_layer_shear(alt_arr,upath,vpath,lati,loni,date_i)
        df_0['mlws'] = copy.deepcopy(np.round(var0.values,4))
        del(var0,alt_arr,upath,vpath)

        # Low Level Specific Humidity
        alt_arr = [800, 1000]
        path = inpath+'SH/SH_'+era5_datestr+'_'+hemi+'.nc'
        var0 = calc_mean_layer_sh(alt_arr,path,lati,loni,date_i)
        #print(var0)
        df_0['llsh'] = copy.deepcopy(np.round(var0*1000,4))
        del(var0,alt_arr,path)      
        
        # Mid Level Specific Humidity
        alt_arr = [400, 800]
        path = inpath+'SH/SH_'+era5_datestr+'_'+hemi+'.nc'
        var0 = calc_mean_layer_sh(alt_arr,path,lati,loni,date_i)
        df_0['mlsh'] = copy.deepcopy(np.round(var0*1000,4))
        del(var0,alt_arr,path)      
               
        # Mid Level Lapse Rate
        alt_arr = [400, 800]
        t_path = inpath+'T/T_'+era5_datestr+'_'+hemi+'.nc'
        z_path = inpath+'GEO/GEO_'+era5_datestr+'_'+hemi+'.nc'
        var0 = calc_mean_lapse_rate(alt_arr,t_path,z_path,lati,loni,date_i)
        df_0['mllr'] = copy.deepcopy(np.round(var0.values,4))
        del(var0,alt_arr,t_path,z_path)      

        # MUCAPE
        path = inpath+'MUCAPE/MUCAPE_'+era5_datestr+'.nc'
        var0 = calc_2D_var(path,lati,loni,date_i)
        df_0['mucape'] = copy.deepcopy(np.round(var0,4))
        del(var0,path)             

        # CIN
        path = inpath+'CIN/CIN_'+era5_datestr+'.nc'
        var0 = calc_2D_var(path,lati,loni,date_i)
        df_0['mlcin'] = copy.deepcopy(np.round(var0,4))
        del(var0,path)             

        # FL
        path = inpath+'FL/FL_'+era5_datestr+'.nc'
        var0 = calc_2D_var(path,lati,loni,date_i)
        df_0['fl'] = copy.deepcopy(np.round(var0,4))
        del(var0,path)             
                
        #print('DFO')
        #print(df_0)
        #print('_________________________________')
        if n == 0:
            data_all = copy.deepcopy(df_0)
        else:
            data_all = pd.concat([data_all,df_0])

file = open(pickpath+filename, 'wb')
# dump information to that file
pickle.dump(data_all, file)
file.close() # close the file            
        


n= 0


/tmp/ipykernel_937/3875488777.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0['lati'] = lati
/tmp/ipykernel_937/3875488777.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0['loni'] = loni


NameError: name 'date_list' is not defined